# MovieLens 1M Dataset Analysis 

## First, read these data files into a dataframe object and merge them 

In [3]:
import pandas as pd
unames=["user_id","gender","age","occupation","zip"]
users=pd.read_table("users.dat",sep="::",header=None,names=unames,engine="python")

In [4]:
users.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [72]:
rnames=["user_id","movie_id","rating","timestamp"]
ratings=pd.read_table("ratings.dat",sep="::",header=None,names=rnames,engine="python")

In [73]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [9]:
mnames=["movie_id","title","genres"]
movies=pd.read_table("movies.dat",sep="::",header=None,names=mnames,engine="python",encoding = "latin-1")

In [10]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


make one dataframe by merging three dataframes

In [11]:
data=pd.merge(pd.merge(ratings,users),movies)

In [12]:
data.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy


### Calculating mean ratings for each movie with gender difference 

In [18]:
mean_ratings=data.pivot_table("rating",index="title",columns="gender",aggfunc="mean")

In [168]:
mean_ratings.head()

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024


### Let’s  filter down to movies that received at least 250 ratings (a completely arbitrary number);


In [13]:
ratings_by_title=data.groupby("title").size()

In [14]:
ratings_by_title.head()

title
$1,000,000 Duck (1971)            37
'Night Mother (1986)              70
'Til There Was You (1997)         52
'burbs, The (1989)               303
...And Justice for All (1979)    199
dtype: int64

get titles that have at least 250 ratings

In [15]:
active_titles=ratings_by_title[ratings_by_title>=250].index

In [16]:
active_titles

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)',
       ...
       'X-Men (2000)', 'Year of Living Dangerously (1982)',
       'Yellow Submarine (1968)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Sherlock Holmes (1985)',
       'Zero Effect (1998)', 'eXistenZ (1999)'],
      dtype='object', name='title', length=1216)

Get mean ratings for these active titles

In [19]:
mean_ratings_active_titles=mean_ratings.loc[active_titles]

In [20]:
mean_ratings_active_titles.head()

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421


Let's see the top movies amongs the female viewers

In [21]:
top_female_ratings=mean_ratings_active_titles.sort_values(by="F",ascending=False)
top_female_ratings.head(10)

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


### Measuring Rating Disagreement 

In [22]:
mean_ratings_active_titles["diff"]=mean_ratings_active_titles["M"]-mean_ratings_active_titles["F"]

In [23]:
mean_ratings_active_titles.sort_values(by="diff",inplace=True)

In [24]:
mean_ratings_active_titles.head()

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777


### Most disagreement among viewers

In [25]:
ratings_std_by_title=data.groupby("title")["rating"].std()

In [26]:
ratings_std_by_title.head()

title
$1,000,000 Duck (1971)           1.092563
'Night Mother (1986)             1.118636
'Til There Was You (1997)        1.020159
'burbs, The (1989)               1.107760
...And Justice for All (1979)    0.878110
Name: rating, dtype: float64

In [27]:
ratings_std_by_active_titles=ratings_std_by_title.loc[active_titles]

In [28]:
ratings_std_by_active_titles.sort_values(ascending=False)[:10]

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64

## Create a dataframe only from active titles

In [31]:
data_reduced=data[data.title.isin(active_titles)];data_reduced.size

8089220

finding how many titles we have in this data_reduced dataframe

In [32]:
data.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy


now find top-rated 10 movies using this data_reduced data frame

In [37]:
# calculate average rating for each movie
top_rated=data_reduced.pivot_table(values="rating",index="title")

In [38]:
top_rated

,rating
title,
"'burbs, The (1989)",2.910891
10 Things I Hate About You (1999),3.422857
101 Dalmatians (1961),3.596460
101 Dalmatians (1996),3.046703
12 Angry Men (1957),4.295455
...,...
Young Guns (1988),3.418149
Young Guns II (1990),2.907859
Young Sherlock Holmes (1985),3.390501


now do the above by finding also the number of rating for each movie

In [39]:
# now we want to both calculate average rating and the number of rating for each movie
top_rated=data_reduced.pivot_table("rating",index="title",aggfunc=["mean","count"]);top_rated.head()

,mean,count
,rating,rating
title,,
"'burbs, The (1989)",2.910891,303
10 Things I Hate About You (1999),3.422857,700
101 Dalmatians (1961),3.596460,565
101 Dalmatians (1996),3.046703,364
12 Angry Men (1957),4.295455,616


In [40]:
top_rated.sort_values(by=("mean","rating"),ascending=False,inplace=True)

In [41]:
top_rated.head()

,mean,count
,rating,rating
title,,
Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954),4.560510,628
"Shawshank Redemption, The (1994)",4.554558,2227
"Godfather, The (1972)",4.524966,2223
"Close Shave, A (1995)",4.520548,657
"Usual Suspects, The (1995)",4.517106,1783


In [42]:
top_rated.columns=top_rated.columns.droplevel(1);top_rated.columns

Index(['mean', 'count'], dtype='object')

In [43]:
top_rated.head()

,mean,count
title,,
Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954),4.560510,628
"Shawshank Redemption, The (1994)",4.554558,2227
"Godfather, The (1972)",4.524966,2223
"Close Shave, A (1995)",4.520548,657
"Usual Suspects, The (1995)",4.517106,1783


## Creating a pivot table for average movie rating for each occupation 

In [45]:
# each line shows a movie and each column shows an occupation
avg_rating_by_occupation=data_reduced.pivot_table("rating",index="title",columns="occupation")

In [46]:
avg_rating_by_occupation.head()

occupation,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",2.580645,2.958333,2.950000,2.833333,3.022222,2.916667,2.625000,2.615385,5.0,2.800000,...,2.000000,2.933333,NaN,3.250000,3.200000,2.833333,3.000000,3.100000,3.400000,3.000000
10 Things I Hate About You (1999),3.337079,3.463415,3.285714,3.478261,3.391026,3.769231,3.428571,3.558140,4.0,3.916667,...,3.200000,3.250000,4.000000,3.388889,3.076923,3.375000,3.510638,3.571429,3.090909,3.052632
101 Dalmatians (1961),3.546667,3.627451,3.625000,3.736842,3.423529,3.666667,3.736842,3.716981,4.0,3.916667,...,4.444444,3.454545,3.142857,3.478261,4.111111,3.521739,3.641026,3.625000,2.857143,3.709677
101 Dalmatians (1996),3.050847,3.117647,2.944444,3.166667,2.595238,3.333333,3.437500,3.190476,3.0,2.909091,...,3.200000,2.916667,3.666667,3.071429,4.000000,3.375000,3.352941,3.400000,3.200000,2.571429
12 Angry Men (1957),4.318841,4.148148,4.333333,4.400000,4.115385,4.400000,4.516129,4.230769,5.0,4.400000,...,4.409091,4.380952,4.368421,4.333333,4.571429,4.366667,4.333333,4.625000,4.428571,4.230769


### Find the top-10 movie that has highest standart deviation by different occupations 

In [193]:
import numpy as np

In [47]:
std_by_occupation=avg_rating_by_occupation.std(axis=1)

In [195]:
std_by_occupation.head()

title
'burbs, The (1989)                   0.694439
10 Things I Hate About You (1999)    0.284100
101 Dalmatians (1961)                0.317771
101 Dalmatians (1996)                0.320831
12 Angry Men (1957)                  0.204460
dtype: float64

now sort this series and find the top-10 movies

In [48]:
std_by_occupation.nlargest(10)

title
Tank Girl (1995)                           0.803774
Immortal Beloved (1994)                    0.802270
Some Folks Call It a Sling Blade (1993)    0.799291
Super Mario Bros. (1993)                   0.779125
Top Hat (1935)                             0.778740
Eat Drink Man Woman (1994)                 0.767672
Blue Lagoon, The (1980)                    0.752676
Story of Us, The (1999)                    0.721351
Limey, The (1999)                          0.720208
'burbs, The (1989)                         0.712480
dtype: float64

## Extract the year information from the title

In [49]:
data_reduced["year"]=data_reduced.title.map(lambda x: x[-5:-1])

C:\Users\mertc\AppData\Local\Temp\ipykernel_18580\3761084441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_reduced["year"]=data_reduced.title.map(lambda x: x[-5:-1])


In [50]:
data_reduced.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,year
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,1975
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,1996
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance,1964
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama,2000
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy,1998


### Find the average rating by movie year 

In [51]:
avg_rating_by_year=data_reduced.pivot_table("rating","year")

In [52]:
avg_rating_by_year.head()

,rating
year,
1925,4.189091
1926,4.082474
1930,4.194030
1931,4.190863
1933,3.900000


sort this table and find the top-10 years having the highest average rating score

In [53]:
avg_rating_by_year.nlargest(10, "rating")

,rating
year,
1949,4.452083
1946,4.302041
1944,4.264686
1934,4.262763
1952,4.246967
1941,4.245691
1957,4.243733
1936,4.236066
1962,4.222766


## Finding top-rated movies at each year

first find average rating by title

In [54]:
avg_rating_by_title=data_reduced.pivot_table("rating",index="title")

In [55]:
avg_rating_by_title.head()

,rating
title,
"'burbs, The (1989)",2.910891
10 Things I Hate About You (1999),3.422857
101 Dalmatians (1961),3.596460
101 Dalmatians (1996),3.046703
12 Angry Men (1957),4.295455


In [56]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


we should add year column

In [57]:
avg_rating_by_title["year"] = data_reduced[["title","year"]].drop_duplicates().set_index("title")

In [58]:
movies["year"]=movies.title.map(lambda x: x[-5:-1])

In [59]:
movies.head()

,movie_id,title,genres,year
0,1,Toy Story (1995),Animation|Children's|Comedy,1995
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [60]:
movies.set_index("title",inplace=True)

In [61]:
movies.head()

,movie_id,genres,year
title,,,
Toy Story (1995),1,Animation|Children's|Comedy,1995
Jumanji (1995),2,Adventure|Children's|Fantasy,1995
Grumpier Old Men (1995),3,Comedy|Romance,1995
Waiting to Exhale (1995),4,Comedy|Drama,1995
Father of the Bride Part II (1995),5,Comedy,1995


now we can add movie year into avg_rating_by_title

In [62]:
avg_rating_by_title["year"]=movies.year

In [63]:
avg_rating_by_title.head()

,rating,year
title,,
"'burbs, The (1989)",2.910891,1989
10 Things I Hate About You (1999),3.422857,1999
101 Dalmatians (1961),3.596460,1961
101 Dalmatians (1996),3.046703,1996
12 Angry Men (1957),4.295455,1957


In [64]:
def find_top_movie(group):
    movie=group.sort_values(by="rating",ascending=False)
    return movie.index[0]


In [65]:
avg_rating_by_title.groupby("year").apply(find_top_movie)[-20:]

C:\Users\mertc\AppData\Local\Temp\ipykernel_18580\3560421524.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_rating_by_title.groupby("year").apply(find_top_movie)[-20:]


year
1981                       Raiders of the Lost Ark (1981)
1982                                  Blade Runner (1982)
1983                            Christmas Story, A (1983)
1984                                       Amadeus (1984)
1985                                           Ran (1985)
1986                                        Aliens (1986)
1987                           Princess Bride, The (1987)
1988                               Cinema Paradiso (1988)
1989                                       Henry V (1989)
1990                             Creature Comforts (1990)
1991                     Silence of the Lambs, The (1991)
1992                              Grand Day Out, A (1992)
1993                              Schindler's List (1993)
1994                     Shawshank Redemption, The (1994)
1995                                Close Shave, A (1995)
1996    Wallace & Gromit: The Best of Aardman Animatio...
1997           Life Is Beautiful (La Vita è bella) (1997)
1998     

In [66]:
def find_top_movie(group):
    movie=group.sort_values(by="rating",ascending=False)
    return pd.Series([movie.index[0],movie.iloc[0]["rating"],len(movie)],
                      index=["title","average_rating","number of movies"])

In [69]:
avg_rating_by_title.groupby("year").apply(find_top_movie)[-20:]

C:\Users\mertc\AppData\Local\Temp\ipykernel_18580\3560421524.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_rating_by_title.groupby("year").apply(find_top_movie)[-20:]


,title,average_rating,number of movies
year,,,
1981,Raiders of the Lost Ark (1981),4.477725,19
1982,Blade Runner (1982),4.273333,26
1983,"Christmas Story, A (1983)",4.238905,17
1984,Amadeus (1984),4.251809,32
1985,Ran (1985),4.268908,35
1986,Aliens (1986),4.125824,37
1987,"Princess Bride, The (1987)",4.303710,28
1988,Cinema Paradiso (1988),4.287805,28
1989,Henry V (1989),4.286385,34
